<a href="https://colab.research.google.com/github/SergeyShchus/Satellite-Imagery-Analysis/blob/master/EarthEngine/Notebook/Managing_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

One of the most commonly asked questions by Earth Engine users is - *How do I download all images in a collection*? The Earth Engine Python API comes with a `ee.batch` module that allows you to launch batch exports and manage tasks. The recommended way to do batch exports like this is to use the Python API's `ee.batch.Export` functions and use a Python for-loop to iterate and export each image. The `ee.batch` module also gives you ability to control *Tasks* - allowing you to automate exports.

In [1]:
import ee

In [2]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=8xTyGypUnqtJXa5XCgA3L-2CG9vNah7hHAO_r_jQJd4&tc=mtRgjml8quRHkfAu85D7Ahg0M-xxOMheN1UhbhSJjNI&cc=IID7jbCslu5Jn7DwuAs7RNbEHBnOS4JQXdD7ND_fBh4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJo1xp_5203JFOhZbv7-zE6augIU6jI4tZ5cMXnVRk0RIuse2DLYb1w

Successfully saved authorization token.


In [3]:
ee.Initialize()

#### Create a Collection

In [4]:
geometry = ee.Geometry.Point([107.61303468448624, 12.130969369851766])
s2 = ee.ImageCollection("COPERNICUS/S2")
rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

# Write a function for Cloud masking
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

filtered = s2 \
  .filter(ee.Filter.date('2019-01-01', '2020-01-01')) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.bounds(geometry)) \
  .map(maskS2clouds)

# Write a function that computes NDVI for an image and adds it as a band
def addNDVI(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('ndvi')
  return image.addBands(ndvi)

withNdvi = filtered.map(addNDVI)

#### Export All Images

Exports are done via the ``ee.batch`` module. A key difference between javascript and Python version is that the `region` parameter needs to be supplied with actual geometry coordinates.

In [5]:
image_ids = withNdvi.aggregate_array('system:index').getInfo()
print('Total images: ', len(image_ids))

Total images:  31


In [6]:
# Export with 100m resolution for this demo
for i, image_id in enumerate(image_ids):
  image = ee.Image(withNdvi.filter(ee.Filter.eq('system:index', image_id)).first())
  task = ee.batch.Export.image.toDrive(**{
    'image': image.select('ndvi'),
    'description': 'Image Export {}'.format(i+1),
    'fileNamePrefix': image.id().getInfo(),
    'folder':'earthengine',
    'scale': 100,
    'region': image.geometry().bounds().getInfo()['coordinates'],
    'maxPixels': 1e10
  })
  task.start()
  print('Started Task: ', i+1)

Started Task:  1
Started Task:  2
Started Task:  3
Started Task:  4
Started Task:  5
Started Task:  6
Started Task:  7
Started Task:  8
Started Task:  9
Started Task:  10
Started Task:  11
Started Task:  12
Started Task:  13
Started Task:  14
Started Task:  15
Started Task:  16
Started Task:  17
Started Task:  18
Started Task:  19
Started Task:  20
Started Task:  21
Started Task:  22
Started Task:  23
Started Task:  24
Started Task:  25
Started Task:  26
Started Task:  27
Started Task:  28
Started Task:  29
Started Task:  30
Started Task:  31


#### Manage Running/Waiting Tasks

You can manage tasks as well. Get a list of tasks and get state information on them

In [7]:
tasks = ee.batch.Task.list()
for task in tasks:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)

QMA2N3LKPMYWNWA4MAZJJYNL READY
WNPZWWPJGAQORLTCNVUNLTTJ READY
MSK27QTHF23DDV6YBRJWNOKF READY
4RPLCTHG2O3XPUY47DT2V4P2 READY
SIWB3BHX3ZKM4KUVS7BZK7TG READY
XHZHV3V25BEQZVL56RPRGMIW READY
OPT2OY2EILVJZ5M2DDYXILWW READY
JQOZ36YFL6CTPAVHSVB5TLXA READY
RUS7YIZHIS3KSCIYQDOW5MT2 READY
VSBGOB5XARG76ZBYQHRQ5Z6C READY
4AOZOLNHKZZVQ3BQ2E4Z4UQP READY
UJQQH3KYOJFN54HVI4MVNKB6 READY
4ILOYXK6ZIXWTGCA7VVTJB4B READY
OSGSRIN7CMYVMTRTNGO3T27T READY
BFMZRJJ5GTFAGLVKZRSQN762 READY
ALVNCBJCHRMSRH7JKJN7R7TK READY
GAUARTAXCW32KUL4VE22WJ4U READY
3WUV3PULX5OCAKLD2NLDDPYZ READY
XZ2FKOC7G5E34IUTFMZWYO5Z READY
BHUFCUPRXN7WBLQ3ZAQQGL2M READY
H2RRSKGLC2MXE5TIXC5YKOOW READY
X2HRU2LLM5BSOUS3ZXRIRDDF READY
BCQVB6TRUB7MKVTL7VDQMO5F READY
VQWWS6UX2W3CFP5LI56RPXMS READY
QRZZRZLMWOOPMBAYYARZF4Z2 READY
YYCEJ3U5D3FR2AH7V4H6TYS2 READY
H3QNA6GTMD3QPYOYLKJ6OR3U READY
BNDALKCKR5TD3JYQN5WMK5ZO READY
3O2EKLLSAJDLMLEAK2BGQBCH RUNNING
PZESR7JGSACIL2CST6FD3XTJ RUNNING
AC2LG7BGMH3HW75FK5T5CJZO RUNNING
NZRDUFUGBWEF4PEYPGEENYRY COMPLETE

You can cancel tasks as well

In [8]:
tasks = ee.batch.Task.list()
for task in tasks:
    task_id = task.status()['id']
    task_state = task.status()['state']
    if task_state == 'RUNNING' or task_state == 'READY':
        task.cancel()
        print('Task {} canceled'.format(task_id))
    else:
        print('Task {} state is {}'.format(task_id, task_state))

Task QMA2N3LKPMYWNWA4MAZJJYNL canceled
Task WNPZWWPJGAQORLTCNVUNLTTJ canceled
Task MSK27QTHF23DDV6YBRJWNOKF canceled
Task 4RPLCTHG2O3XPUY47DT2V4P2 canceled
Task SIWB3BHX3ZKM4KUVS7BZK7TG canceled
Task XHZHV3V25BEQZVL56RPRGMIW canceled
Task OPT2OY2EILVJZ5M2DDYXILWW canceled
Task JQOZ36YFL6CTPAVHSVB5TLXA canceled
Task RUS7YIZHIS3KSCIYQDOW5MT2 canceled
Task VSBGOB5XARG76ZBYQHRQ5Z6C canceled
Task 4AOZOLNHKZZVQ3BQ2E4Z4UQP canceled
Task UJQQH3KYOJFN54HVI4MVNKB6 canceled
Task 4ILOYXK6ZIXWTGCA7VVTJB4B canceled
Task OSGSRIN7CMYVMTRTNGO3T27T canceled
Task BFMZRJJ5GTFAGLVKZRSQN762 canceled
Task ALVNCBJCHRMSRH7JKJN7R7TK canceled
Task GAUARTAXCW32KUL4VE22WJ4U canceled
Task 3WUV3PULX5OCAKLD2NLDDPYZ canceled
Task XZ2FKOC7G5E34IUTFMZWYO5Z canceled
Task BHUFCUPRXN7WBLQ3ZAQQGL2M canceled
Task H2RRSKGLC2MXE5TIXC5YKOOW canceled
Task X2HRU2LLM5BSOUS3ZXRIRDDF canceled
Task BCQVB6TRUB7MKVTL7VDQMO5F canceled
Task VQWWS6UX2W3CFP5LI56RPXMS canceled
Task QRZZRZLMWOOPMBAYYARZF4Z2 canceled
Task YYCEJ3U5D3FR2AH7V4H6

### Exercise

The code below uses the TerraClimate data and creates an ImageCollection with 12 monthly images of maximum temperature. It also extract the geometry for Australia from the LSIB collection. Add the code to start an export task for each image in the collection for australia.

- **Hint1**: TerraClimate images have a scale of 4638.3m
- **Hint2**: You need to export the image contained in the clippedImage variable

In [9]:
import ee

lsib = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
australia = lsib.filter(ee.Filter.eq('country_na', 'Australia'))
geometry = australia.geometry()

terraclimate = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')
tmax = terraclimate.select('tmmx')

def scale(image):
  return image.multiply(0.1) \
    .copyProperties(image,['system:time_start'])

tmaxScaled = tmax.map(scale)

filtered = tmaxScaled \
  .filter(ee.Filter.date('2020-01-01', '2021-01-01')) \
  .filter(ee.Filter.bounds(geometry))

image_ids = filtered.aggregate_array('system:index').getInfo()
print('Total images: ', len(image_ids))

Total images:  12


Replace the comments with your code.

In [10]:
for i, image_id in enumerate(image_ids):
    exportImage = ee.Image(filtered.filter(ee.Filter.eq('system:index', image_id)).first())
    # Clip the image to the region geometry
    clippedImage = exportImage.clip(geometry)
    
    ## Create the export task using ee.batch.Export.image.toDrive()
    
    ## Start the task

# Новый раздел

# Новый раздел

In [ ]:
from google.colab import drive
drive.mount('/content/drive')